<a href="https://colab.research.google.com/github/YoTaSo/datascience/blob/main/deeplearning/lstm_gru_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Initial setups and imports

In [ ]:
import torch
import torch.nn as nn
import numpy as np
import gzip
import os
from torch.utils.data import DataLoader
from torch import optim
from google.colab import drive
from preprocess import TextDataset
import sentencepiece as spm
import nltk
from nltk import tokenize
from collections import namedtuple
from torch.utils.tensorboard import SummaryWriter

In [ ]:

drive.mount('/content/gdrive')
root_path = 'gdrive/My Drive/AMAL_TP6'
os.chdir(root_path)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [ ]:
with open('trump_full_speech.txt', 'r') as f:
  text = f.read()
#text = text.split()
#!pip install nltk

In [ ]:
!pip install sentencepiece
!pip install  git+https://github.com/experimaestro/datamaestro git+https://github.com/experimaestro/datamaestro_image git+https://github.com/experimaestro/datamaestro_ml git+https://github.com/experimaestro/datamaestro_text
!pip install -U PyYAML


# Models

In [ ]:

class LSTMspeech(torch.nn.Module):
  def __init__(self,latent=70,embdDim=100,dimout=2000):
    super().__init__()
    self.embedding = nn.Embedding(dimout,embdDim,padding_idx=2)
    self.latent=latent
    #self.dim=dim
    #if multiDim: self.dim=x.size()[1]
    #latent=#arg

    self.oubliLinear=nn.Linear(latent+embdDim, latent)
    self.oublActF=nn.Sigmoid()

    self.entreeLinear=nn.Linear(latent+embdDim, latent)
    self.entreeActF=nn.Sigmoid()

    self.interneLayer=nn.Linear(latent+embdDim, latent)
    #self.interneActF=torch.tanh()

    self.sortieLinear=nn.Linear(latent+embdDim, latent)
    self.sortieActF=nn.Sigmoid()

    self.decoder = nn.Linear(latent, dimout)
    self.actF=nn.Softmax(dim=1)

  def forward(self,x):
    outsLis=[]
    lastH=torch.zeros(x.size()[0], self.latent,dtype=torch.float)
    lastC=torch.zeros(x.size()[0], self.latent,dtype=torch.float)
    for i in range(0,x.size()[1]):
      inp=x[:,i]
      inp=self.embedding(inp)# batch_size*embd_dim_out

      inp=torch.cat((inp,lastH), dim=1).float()#batch_size*(embd_dim_out+latent)

      oubli=self.oubliLinear(inp)#batch_size*latent
      oubli=self.oublActF(oubli)

      entree=self.entreeLinear(inp)#batch_size*latent
      entree=self.entreeActF(entree)

      interne=self.interneLayer(inp)#batch_size*latent
      lastC=oubli*lastC + entree*torch.tanh(interne)#self.interneActF(interne)

      sortie=self.sortieLinear(inp)#batch_size*latent
      sortie=self.sortieActF(sortie)
      
      lastH=sortie*torch.tanh(lastC)

      outp=self.decoder(lastH)
      outp=self.actF(outp)
      outsLis.append(outp)
    out=torch.stack(outsLis, dim=2)
    return out

In [ ]:

class GRUspeech(torch.nn.Module):
  def __init__(self,latent=70,embdDim=100,dimout=2000):
    super().__init__()
    self.embedding = nn.Embedding(2000,embdDim,padding_idx=2)
    self.latent=latent
    #self.dim=dim
    #if multiDim: self.dim=x.size()[1]
    #latent=#arg

    self.zLinear=nn.Linear(latent+embdDim, latent,bias=False)
    self.zActF=nn.Sigmoid()

    self.rLinear=nn.Linear(latent+embdDim, latent,bias=False)
    self.rActF=nn.Sigmoid()

    self.hLayer=nn.Linear(latent+embdDim, latent,bias=False)
    #self.interneActF=torch.tanh()

    self.decoder = nn.Linear(latent, dimout)
    self.actF=nn.Softmax(dim=1)

  def forward(self,x):
    
    lastH=torch.zeros(x.size()[0], self.latent,dtype=torch.float)#=500*30
    #lastC=torch.zeros(x.size()[0], self.latent,dtype=torch.float)
    outsLis=[]
    for i in range(0,x.size()[1]):
      inp=x[:,i]
      embdinp=self.embedding(inp)# batch_size*embd_dim_out=500*100

      inp=torch.cat((embdinp,lastH), dim=1).float()#batch_size*(embd_dim_out+latent)500*130

      zz=self.zLinear(inp)#batch_size*latent= 500*30
      zz=self.zActF(zz)

      rr=self.rLinear(inp)#batch_size*latent=500*30
      rr=self.rActF(rr)

      inpHh=torch.cat((embdinp,rr*lastH), dim=1).float()#500*
      #print(inpHh.size())
      hh=self.hLayer(inpHh)#batch_size*latent
      lastH=(1-zz)*lastH + zz*torch.tanh(hh)#self.interneActF(interne)

      outp=self.decoder(lastH)
      outp=self.actF(outp)
      outsLis.append(outp)
    out=torch.stack(outsLis, dim=2)
    return out

In [ ]:

class RNNspeech(torch.nn.Module):
  def __init__(self,latent=15,embdDim=100,dimout=2000):
    super().__init__()
    self.embedding = nn.Embedding(dimout, embdDim)
    self.latent=latent
    self.singleLayer = nn.Linear(latent+embdDim, latent)
    self.decoder = nn.Linear(latent, dimout)
    self.actF=nn.Sigmoid()#nn.Softmax(dim=1)

  def forward(self,x):
    
    lastH=torch.zeros(x.size()[0], self.latent,dtype=torch.float)
    outsLis=[]
    for i in range(0,x.size()[1]):
      inp=x[:,i]
      inp=self.embedding(inp)
      inp=torch.cat((inp,lastH), dim=1).float()
      inp=self.singleLayer(inp)
      inp = torch.tanh(inp)
      lastH=inp.float()
      outp=self.decoder(lastH)
      outp=self.actF(outp)
      outsLis.append(outp)
    out=torch.stack(outsLis, dim=2).squeeze()
    return out

# preparing data

In [ ]:
#!pip install sentencepiece
vocab_size=1000

spm.SentencePieceTrainer.train('--input=myfile.txt --model_prefix=m --vocab_size=1000 --unk_id=0 --bos_id=-1 --eos_id=1 --pad_id=2')

True

In [ ]:

# makes segmenter instance and loads the model file (m.model)
sp = spm.SentencePieceProcessor()
sp.load('m.model')

# encode: text => id
print(sp.encode_as_pieces('This is a test'))
print(sp.encode_as_ids('This is a test'))

# decode: id => text
print(sp.decode_pieces(['▁This', '▁is', '▁a', '▁t', 'est']))
print(sp.decode_ids([156, 22, 11, 248, 277]))

['▁This', '▁is', '▁a', '▁t', 'est']
[215, 34, 13, 208, 238]
This is a test
butd and were Our


In [ ]:
encoded_text=sp.encode_as_ids(text)
encoded_text_tensor=torch.tensor(encoded_text)

In [ ]:

Batch = namedtuple("Batch", "text")
class myTextDataset(torch.utils.data.Dataset):
    
    def __init__(self, text: torch.LongTensor, sizes: torch.LongTensor):
        self.text = text
        self.sizes = sizes
        #self.labels = labels
        
    def __len__(self):
        return len(self.sizes)

    def __getitem__(self, index: int):
        return self.text[self.sizes[index]:self.sizes[index+1]]

    @staticmethod
    def collate(batch):
        data = [item for item in batch]
        return Batch(torch.nn.utils.rnn.pad_sequence(data, batch_first=True))

In [ ]:

nltk.download('punkt')

SentenceSplt=tokenize.sent_tokenize(text)
ssizes=np.array([0]+[len(sp.encode_as_ids(x)) for x in SentenceSplt])
ssizes=torch.from_numpy(np.cumsum(ssizes))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
textDatasetObj=myTextDataset(encoded_text_tensor,ssizes)


In [ ]:
train_loader = DataLoader(textDatasetObj, collate_fn=myTextDataset.collate, shuffle=True, batch_size=400)
#test_loader  = DataLoader( test_ds, collate_fn=myTextDataset.collate, shuffle=True, batch_size=400)

# fiitng DNNs

In [ ]:
#batch learning

#torch.manual_seed(42)

writer = SummaryWriter()
vocab_size=1000
#encodedSpech=string2code(text)
# Now we can create a model and send it at once to the device
model1= LSTMspeech(15,100,vocab_size)
model2=GRUspeech(15,100,vocab_size)
model3=RNNspeech(15,100,vocab_size)

lr = 0.0005
n_epochs = 100
loss_fn=torch.nn.CrossEntropyLoss( )
optimizer1=torch.optim.Adam(model1.parameters(), lr=lr)
optimizer2=torch.optim.Adam(model2.parameters(), lr=lr)
optimizer3=torch.optim.Adam(model2.parameters(), lr=lr)
for t in range(n_epochs):
  for X in train_loader:

    #X=dataLoader(400)
    n=X.text.size()[1]

    output1 = model1(X.text[:,0:n-1])
    output2 = model2(X.text[:,0:n-1])
    output3 = model3(X.text[:,0:n-1])

    loss1 = loss_fn(output1,X.text[:,1:n])
    loss2 = loss_fn(output2,X.text[:,1:n])
    loss3 = loss_fn(output3,X.text[:,1:n])
    #if t%100==0:
    print(t,"lstm: ",loss1.item()," GRU: ",loss2.item()," Rnn: ",loss3.item())

    
    loss1.backward()
    loss2.backward()
    loss3.backward()
    #print(t,model.singleLayer.weight.grad)
    optimizer1.step()
    optimizer2.step()
    optimizer3.step()

    optimizer1.zero_grad()
    optimizer2.zero_grad()
    optimizer3.zero_grad()

    writer.add_scalars('Run2',{'LSTM':loss1.item(),
                                'GRU':loss2.item(),
                                'RNN':loss3.item()}, t)


In [ ]:
%load_ext tensorboard
%tensorboard --logdir=runs